# Hello World Example

The goal of this notebook is to provide a hello world example of using Kaskada hosted as a Binder notebook. The notebook is designed to run from top to bottom and explain the fundamentals of Kaskada. In this notebook, we will cover:

* Local session with a local builder.
* Creating and loading data to a table.
* Running queries 
* Creating views

*Last modified on 3/6/2023.*

## Local Session

A local session is created by using the `LocalBuilder` object as part of the `kaskada.api.session` module. By default, the builder will fetch the latest release from GitHub and attempt to download/run the correct system binaries. There is a basic caching mechanism designed to reuse existing downloads.

### TODO: Remove once public

The following step is required because the Kaskada repo is not public yet. Once public, we can remove this step and no additional authentication is required.

In [ ]:
import kaskada.api.release as release
import os
from getpass import getpass

os.environ[release.ReleaseClient.GITHUB_ACCESS_TOKEN_ENV] = getpass(prompt='Github Access Token:')

In [ ]:
from kaskada.api.session import LocalBuilder
session = LocalBuilder().build()

## Example Data

Kaskada is able to read Parquet and CSV data. In this example, we will read a CSV file included called `dataset1.csv`.

In [ ]:
import pandas as pd
pd.read_csv('dataset1.csv')

## Create a Table

Data is loaded to Kaskada through a Table. To create a table, you will need the entity key and time column names. The Kaskada python client follows a service module separation so all table related operations are under the `table` module.

In [ ]:
import kaskada.table
kaskada.table.create_table(table_name='transactions', entity_key_column_name='id', time_column_name='time')

## Load Data to a Table

Data is loaded to a table through the `load` method. Currently, we accept CSV or Parquet data.

In [ ]:
kaskada.table.load('transactions', 'dataset1.csv')

## See all your tables

Tables are a managed resource and can be seen in detail by listing or getting them.

In [ ]:
kaskada.table.list_tables()

In [ ]:
kaskada.table.get_table('transactions')

## Start Feature Engineering

Once your data is in a table, you can begin the journey of feature engineering using our magic extension `fenlmagic`. To load the extension: `%load_ext fenlmagic`.

In [ ]:
%load_ext fenlmagic

## Run some queries

Queries are ran as multi-line cell magics with `%%fenl`. Here are some examples:

In [ ]:
%%fenl
{
  last_transaction: last(transactions.amount)
}

## Queries as variables

The results of queries can be saved to a variable for subsequent use by using the `--var` parameter.

In [ ]:
%%fenl --var my_query
{
  last_transaction: last(transactions.amount)
}

In [ ]:
# Get the original query used
my_query.query

## See previous queries

The metadata used for a query is stored and can be retrieved using a list queries call.

In [ ]:
import kaskada.query
kaskada.query.list_queries()

## Views

Views are named fenl expressions that can be referenced in other fenl expressions or in materializations.

In [ ]:
import kaskada.view
kaskada.view.create_view(view_name = 'my_first_view', expression="last(transactions.amount)")

In [ ]:
# Create a view from an existing query
previous_query = kaskada.query.list_queries().queries[0]
previous_query
kaskada.view.create_view(view_name = "my_second_view", expression=previous_query.expression)

## Views in Queries

A view can be referenced by name in a query

In [ ]:
%%fenl
my_second_view

In [ ]:
%%fenl
my_first_view